In [1]:
%%capture
!pip install pyspark
!pip install delta-spark

In [2]:
import pyspark
from delta import *

In [4]:
%%capture
builder = (pyspark.sql.SparkSession.builder.appName("model_SQL")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [5]:
%%bash
if [ -d spark-warehouse ]
then
    rm -r spark-warehouse
fi

In [6]:
spark.sql("CREATE DATABASE IF NOT EXISTS empresa;")

DataFrame[]

In [7]:
spark.sql(
"""
CREATE OR REPLACE TABLE empresa.funcionario (
  cpf STRING,
  data_nascimento TIMESTAMP,
  primeiro_nome STRING,
  nome_meio STRING,
  ultimo_nome STRING,
  endereco  STRING,
  salario DOUBLE,
  sexo STRING,
  cpf_supervisor STRING,
  numero_departamento INT

) USING DELTA
"""
)

DataFrame[]

In [8]:
spark.sql(
"""
CREATE OR REPLACE TABLE empresa.dependente (
  cpf_funcionario STRING,
  nome_dependente STRING,
  sexo STRING,
  data_nascimento TIMESTAMP,
  parentesco STRING
) USING DELTA
"""
)

DataFrame[]

In [9]:
spark.sql(
"""
INSERT OVERWRITE empresa.funcionario VALUES
    ('65329865388', '1962-12-31','Ricardo','K','Marini','Rua Itapira 44 Santos, SP', 3700, 'M','65329865388',1),
    ('59310458637', '1992-07-14','Vinicius','S','Vale','Rua Artur Bernanrdes 2, SP', 5500, 'M','65329865388',1);
"""
)

DataFrame[]

In [10]:
df = spark.sql("""
SELECT * FROM empresa.funcionario
""")

In [11]:
df.printSchema()

root
 |-- cpf: string (nullable = true)
 |-- data_nascimento: timestamp (nullable = true)
 |-- primeiro_nome: string (nullable = true)
 |-- nome_meio: string (nullable = true)
 |-- ultimo_nome: string (nullable = true)
 |-- endereco: string (nullable = true)
 |-- salario: double (nullable = true)
 |-- sexo: string (nullable = true)
 |-- cpf_supervisor: string (nullable = true)
 |-- numero_departamento: integer (nullable = true)



In [12]:
df.to_pandas_on_spark()

22/05/22 11:30:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/22 11:30:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


,cpf,data_nascimento,primeiro_nome,nome_meio,ultimo_nome,endereco,salario,sexo,cpf_supervisor,numero_departamento
0,65329865388,1962-12-31,Ricardo,K,Marini,"Rua Itapira 44 Santos, SP",3700.0,M,65329865388,1
1,59310458637,1992-07-14,Vinicius,S,Vale,"Rua Artur Bernanrdes 2, SP",5500.0,M,65329865388,1


In [13]:
spark.sql(
"""
CREATE OR REPLACE TABLE empresa.departamento (
  numero_departamento INT,
  data_inicio_gerente TIMESTAMP,
  nome_departamento STRING,
  cpf_gerente STRING
) USING DELTA
"""
)

DataFrame[]

In [14]:
spark.sql(
"""
INSERT OVERWRITE empresa.funcionario VALUES
('3248493205','1966-03-04','Pedro','P','Machado','Rua Projetada A numero 27',2000,'M','65329865388',1),
('8859483743','1973-05-16','Maria','R','Oliveira','Rua Altamira 23',2200,'F','8859483743',2),
('65477565447','1983-10-18','Marcio','J','Gomes','Rua Casemiro 28',1500,'M','3248493205',1)
"""
)

DataFrame[]

In [15]:
spark.sql(
"""
INSERT OVERWRITE empresa.departamento VALUES
(1,'2020-02-01','Pesquisa','65329865388'),
(2,'2018-10-02','Comercial','8859483743')
"""
)

DataFrame[]

In [16]:
spark.sql(
"""
UPDATE empresa.funcionario SET nome_meio = 'S' WHERE cpf = '8859483743';
"""
)

DataFrame[]

In [17]:
funcionario = spark.sql("""
SELECT * FROM empresa.funcionario
""")

In [18]:
funcionario.to_pandas_on_spark()

22/05/22 11:30:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/22 11:30:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


,cpf,data_nascimento,primeiro_nome,nome_meio,ultimo_nome,endereco,salario,sexo,cpf_supervisor,numero_departamento
0,3248493205,1966-03-04,Pedro,P,Machado,Rua Projetada A numero 27,2000.0,M,65329865388,1
1,8859483743,1973-05-16,Maria,S,Oliveira,Rua Altamira 23,2200.0,F,8859483743,2
2,65477565447,1983-10-18,Marcio,J,Gomes,Rua Casemiro 28,1500.0,M,3248493205,1


In [19]:
spark.sql(
"""
UPDATE empresa.funcionario SET salario = 1.1 * salario WHERE numero_departamento = 1;
"""
)

DataFrame[]

In [20]:
funcionario = spark.sql("""
SELECT * FROM empresa.funcionario
""")
funcionario.to_pandas_on_spark()

22/05/22 11:30:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/22 11:30:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


,cpf,data_nascimento,primeiro_nome,nome_meio,ultimo_nome,endereco,salario,sexo,cpf_supervisor,numero_departamento
0,3248493205,1966-03-04,Pedro,P,Machado,Rua Projetada A numero 27,2200.0,M,65329865388,1
1,8859483743,1973-05-16,Maria,S,Oliveira,Rua Altamira 23,2200.0,F,8859483743,2
2,65477565447,1983-10-18,Marcio,J,Gomes,Rua Casemiro 28,1650.0,M,3248493205,1


In [21]:
spark.sql(
"""
DELETE FROM empresa.funcionario WHERE cpf = '65477565447';
"""
)

DataFrame[]

In [22]:
funcionario = spark.sql("""
SELECT * FROM empresa.funcionario
""")
funcionario.to_pandas_on_spark()

22/05/22 11:30:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/05/22 11:30:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


,cpf,data_nascimento,primeiro_nome,nome_meio,ultimo_nome,endereco,salario,sexo,cpf_supervisor,numero_departamento
0,3248493205,1966-03-04,Pedro,P,Machado,Rua Projetada A numero 27,2200.0,M,65329865388,1
1,8859483743,1973-05-16,Maria,S,Oliveira,Rua Altamira 23,2200.0,F,8859483743,2


In [23]:
spark.sql(
"""
INSERT OVERWRITE empresa.departamento VALUES 
(5,'1988-05-22','Pesquisa','33344555587'),
(4,'1995-01-01','Administração','98765432168'),
(1,'1981-06-19','Matriz','88866555576');
"""
)

DataFrame[]

In [24]:
spark.sql("""
INSERT OVERWRITE empresa.dependente VALUES
('33344555587','Alicia','F','1986-04-05','Filha'),
('33344555587','Tiago','M','1983-10-25','Filho'),
('33344555587','Janaína','F','1958-05-03','Esposa'),
('98765432168','Antonio','M','1942-02-18','Marido'),
('12345678966','Michael','M','1988-04-01','Filho'),
('12345678966','Alicia','F','1988-12-30','Filha'),
('12345678966','Elizabeth','F','1967-05-05','Esposa');
""")

DataFrame[]

In [25]:
spark.sql("""
INSERT OVERWRITE empresa.funcionario VALUES
('12345678966','1965-09-01','João','B','Silva','Rua das Flores 751 São Paulo SP','30000','M','33344555587','5'),
('33344555587','1955-08-12','Fernando','T','Wong','Rua da Lapa 34 São Paulo SP','40000','M','88866555576','5'),
('99988777767','1968-01-19','Alice','J','Zelaya','Rua Souza Lima 35 Curitiba PR','25000','F','98765432168','4'),
('98765432168','1941-06-20','Jeniffer','S','Souza','Av Arthur de Lima 54 Santo André','43000','F','88866555576','4'),
('66688444476','1962-09-15','Ronaldo','K','Lima','Rua Rebouças 65 Piracicaba SP','38000','M','33344555587','5'),
('45345345376','1972-07-31','Joice','A','Leite','Av. Lucas Obes 74 São Paulo SP','25000','F','33344555587','5'),
('98798798733','1969-03-29','André','V','Pereira','Rua Timbira 35 São Paulo SP','25000','M','98765432168','4'),
('88866555576','1937-11-10','Jorge','E','Brito','Rua do Horto 35 Sáo Paulo SP','55000','M','','1')
""")

DataFrame[]

In [26]:
spark.sql("""
CREATE OR REPLACE TABLE empresa.localizacao_departamento (
  numero_departamento INT,
  local STRING
) USING DELTA
""")

DataFrame[]

In [27]:
spark.sql("""
INSERT OVERWRITE empresa.localizacao_departamento VALUES
(1,'São Paulo'),(4,'Mauá'),(5,'Santo André'),(5,'Itu'),(5,'São Paulo');
""")

DataFrame[]

In [28]:
spark.sql("""
CREATE OR REPLACE TABLE empresa.projeto (
  nome_projeto STRING,
  numero_projeto STRING,
  local_projeto STRING,
  numero_departamento STRING
) USING DELTA
""")

DataFrame[]

In [29]:
spark.sql("""
INSERT OVERWRITE empresa.projeto VALUES
('Reorganização','20','São Paulo','1'),
('Produto Y','2','Itu','5'),
('Novos Benefícios','30','Mauá','4'),
('Produto X','1','Santo André','4'),
('Informatização','10','Mauá','4'),
('Produto Z','3','São Paulo','5');
""")

DataFrame[]

In [30]:
spark.sql("""
CREATE OR REPLACE TABLE empresa.trabalha (
  cpf_funcionario STRING,
  numero_projeto STRING,
  horas DOUBLE
) USING DELTA
""")

DataFrame[]

In [31]:
spark.sql("""
INSERT OVERWRITE empresa.trabalha VALUES
('12345678966','1',32.5),
('12345678966','2',7.5),
('66688444476','3',40),
('45345345376','1',20),
('45345345376','2',20),
('33344555587','2',10),
('33344555587','3',10),
('33344555587','10',10),
('33344555587','20',10),
('99988777767','30',30),
('99988777767','10',10),
('98798798733','10',35),
('98798798733','30',5),
('98765432168','30',5),
('98765432168','20',20),
('88866555576','20',0);
""")

DataFrame[]

In [32]:
spark.sql("""
SELECT * FROM empresa.projeto
""").show()

+----------------+--------------+-------------+-------------------+
|    nome_projeto|numero_projeto|local_projeto|numero_departamento|
+----------------+--------------+-------------+-------------------+
|   Reorganização|            20|    São Paulo|                  1|
|Novos Benefícios|            30|         Mauá|                  4|
|  Informatização|            10|         Mauá|                  4|
|       Produto X|             1|  Santo André|                  4|
|       Produto Z|             3|    São Paulo|                  5|
|       Produto Y|             2|          Itu|                  5|
+----------------+--------------+-------------+-------------------+



In [33]:
spark.sql("""
SELECT nome_projeto, numero_projeto FROM empresa.projeto
""").show()

+----------------+--------------+
|    nome_projeto|numero_projeto|
+----------------+--------------+
|   Reorganização|            20|
|Novos Benefícios|            30|
|  Informatização|            10|
|       Produto X|             1|
|       Produto Z|             3|
|       Produto Y|             2|
+----------------+--------------+



In [34]:
spark.sql("""
SELECT * FROM empresa.projeto
where local_projeto = 'Mauá'
""").show()

+----------------+--------------+-------------+-------------------+
|    nome_projeto|numero_projeto|local_projeto|numero_departamento|
+----------------+--------------+-------------+-------------------+
|Novos Benefícios|            30|         Mauá|                  4|
|  Informatização|            10|         Mauá|                  4|
+----------------+--------------+-------------+-------------------+



In [35]:
spark.sql("""
SELECT * FROM empresa.projeto
where local_projeto = 'Mauá'
""").show()

+----------------+--------------+-------------+-------------------+
|    nome_projeto|numero_projeto|local_projeto|numero_departamento|
+----------------+--------------+-------------+-------------------+
|Novos Benefícios|            30|         Mauá|                  4|
|  Informatização|            10|         Mauá|                  4|
+----------------+--------------+-------------+-------------------+



In [36]:
spark.sql("""
SELECT * FROM empresa.projeto
where local_projeto = 'Mauá'
and numero_departamento == 4
""").show()

+----------------+--------------+-------------+-------------------+
|    nome_projeto|numero_projeto|local_projeto|numero_departamento|
+----------------+--------------+-------------+-------------------+
|Novos Benefícios|            30|         Mauá|                  4|
|  Informatização|            10|         Mauá|                  4|
+----------------+--------------+-------------+-------------------+



In [37]:
spark.sql("""
SELECT * FROM empresa.funcionario
where data_nascimento  > '1965-01-01'
""").show()

+-----------+-------------------+-------------+---------+-----------+--------------------+-------+----+--------------+-------------------+
|        cpf|    data_nascimento|primeiro_nome|nome_meio|ultimo_nome|            endereco|salario|sexo|cpf_supervisor|numero_departamento|
+-----------+-------------------+-------------+---------+-----------+--------------------+-------+----+--------------+-------------------+
|12345678966|1965-09-01 00:00:00|         João|        B|      Silva|Rua das Flores 75...|30000.0|   M|   33344555587|                  5|
|45345345376|1972-07-31 00:00:00|        Joice|        A|      Leite|Av. Lucas Obes 74...|25000.0|   F|   33344555587|                  5|
|98798798733|1969-03-29 00:00:00|        André|        V|    Pereira|Rua Timbira 35 Sã...|25000.0|   M|   98765432168|                  4|
|99988777767|1968-01-19 00:00:00|        Alice|        J|     Zelaya|Rua Souza Lima 35...|25000.0|   F|   98765432168|                  4|
+-----------+--------------

In [38]:
spark.sql("""
SELECT * FROM empresa.funcionario
where year(data_nascimento)  > 1965
""").show()

+-----------+-------------------+-------------+---------+-----------+--------------------+-------+----+--------------+-------------------+
|        cpf|    data_nascimento|primeiro_nome|nome_meio|ultimo_nome|            endereco|salario|sexo|cpf_supervisor|numero_departamento|
+-----------+-------------------+-------------+---------+-----------+--------------------+-------+----+--------------+-------------------+
|45345345376|1972-07-31 00:00:00|        Joice|        A|      Leite|Av. Lucas Obes 74...|25000.0|   F|   33344555587|                  5|
|98798798733|1969-03-29 00:00:00|        André|        V|    Pereira|Rua Timbira 35 Sã...|25000.0|   M|   98765432168|                  4|
|99988777767|1968-01-19 00:00:00|        Alice|        J|     Zelaya|Rua Souza Lima 35...|25000.0|   F|   98765432168|                  4|
+-----------+-------------------+-------------+---------+-----------+--------------------+-------+----+--------------+-------------------+



In [39]:
spark.sql("""
SELECT cpf FROM empresa.funcionario where numero_departamento  = 5
UNION
SELECT distinct cpf_supervisor FROM empresa.funcionario where numero_departamento  = 5
""").show()

+-----------+
|        cpf|
+-----------+
|12345678966|
|45345345376|
|66688444476|
|33344555587|
|88866555576|
+-----------+



In [40]:
spark.sql("""
SELECT a.primeiro_nome, a.ultimo_nome, a.cpf,
b.cpf_funcionario, b.nome_dependente, b.sexo, b.data_nascimento, b.parentesco
FROM empresa.funcionario a 
inner join empresa.dependente b on a.cpf = b.cpf_funcionario
where a.sexo  = 'F'
""").show()

+-------------+-----------+-----------+---------------+---------------+----+-------------------+----------+
|primeiro_nome|ultimo_nome|        cpf|cpf_funcionario|nome_dependente|sexo|    data_nascimento|parentesco|
+-------------+-----------+-----------+---------------+---------------+----+-------------------+----------+
|     Jeniffer|      Souza|98765432168|    98765432168|        Antonio|   M|1942-02-18 00:00:00|    Marido|
+-------------+-----------+-----------+---------------+---------------+----+-------------------+----------+



In [41]:
spark.sql("""
SELECT a.primeiro_nome, a.nome_meio, a.ultimo_nome,b.nome_projeto
FROM empresa.funcionario a
inner join empresa.departamento c on c.cpf_gerente = a.cpf
inner join empresa.projeto b on b.numero_departamento = c.numero_departamento
""").show()

+-------------+---------+-----------+----------------+
|primeiro_nome|nome_meio|ultimo_nome|    nome_projeto|
+-------------+---------+-----------+----------------+
|     Jeniffer|        S|      Souza|       Produto X|
|     Jeniffer|        S|      Souza|  Informatização|
|     Jeniffer|        S|      Souza|Novos Benefícios|
|     Fernando|        T|       Wong|       Produto Y|
|     Fernando|        T|       Wong|       Produto Z|
|        Jorge|        E|      Brito|   Reorganização|
+-------------+---------+-----------+----------------+



In [42]:
spark.sql("""
SELECT a.primeiro_nome, a.nome_meio, a.ultimo_nome
FROM empresa.funcionario a
INNER JOIN empresa.trabalha b ON a.cpf = b.cpf_funcionario
WHERE a.numero_departamento = 5 AND b.numero_projeto = 10 AND b.horas >= 10
""").show()

+-------------+---------+-----------+
|primeiro_nome|nome_meio|ultimo_nome|
+-------------+---------+-----------+
|     Fernando|        T|       Wong|
+-------------+---------+-----------+



In [43]:
spark.sql("""
SELECT a.primeiro_nome,a.nome_meio,a.ultimo_nome, a.endereco
from empresa.funcionario a
inner join empresa.departamento b on a.numero_departamento = b.numero_departamento
where b.nome_departamento = 'Pesquisa'
""").show()

+-------------+---------+-----------+--------------------+
|primeiro_nome|nome_meio|ultimo_nome|            endereco|
+-------------+---------+-----------+--------------------+
|         João|        B|      Silva|Rua das Flores 75...|
|        Joice|        A|      Leite|Av. Lucas Obes 74...|
|      Ronaldo|        K|       Lima|Rua Rebouças 65 P...|
|     Fernando|        T|       Wong|Rua da Lapa 34 Sã...|
+-------------+---------+-----------+--------------------+



Para cada projeto localizado em 'Mauá' liste o número do projeto, o número do departamento que o controla e o último nome, endereço e data de nascimento do gerente do departamento

In [44]:
spark.sql("""
SELECT b.numero_projeto, b.numero_departamento, a.ultimo_nome, a.endereco, a.data_nascimento
FROM empresa.projeto b
INNER JOIN empresa.departamento c on c.numero_departamento = b.numero_departamento
INNER JOIN empresa.funcionario a on a.cpf = c.cpf_gerente
where b.local_projeto = 'Mauá'
""").show()

+--------------+-------------------+-----------+--------------------+-------------------+
|numero_projeto|numero_departamento|ultimo_nome|            endereco|    data_nascimento|
+--------------+-------------------+-----------+--------------------+-------------------+
|            10|                  4|      Souza|Av Arthur de Lima...|1941-06-20 00:00:00|
|            30|                  4|      Souza|Av Arthur de Lima...|1941-06-20 00:00:00|
+--------------+-------------------+-----------+--------------------+-------------------+



Descobrir os nomes dos funcionarios que trabalham em todos os projetos controlados pelo departamento numero 5

In [45]:
spark.sql("""
SELECT * FROM empresa.projeto where numero_departamento = 5
""").show()

+------------+--------------+-------------+-------------------+
|nome_projeto|numero_projeto|local_projeto|numero_departamento|
+------------+--------------+-------------+-------------------+
|   Produto Z|             3|    São Paulo|                  5|
|   Produto Y|             2|          Itu|                  5|
+------------+--------------+-------------+-------------------+



In [46]:
spark.sql("""
SELECT * FROM empresa.trabalha where numero_projeto in (SELECT distinct numero_projeto FROM empresa.projeto where numero_departamento = 5)
""").show()

+---------------+--------------+-----+
|cpf_funcionario|numero_projeto|horas|
+---------------+--------------+-----+
|    33344555587|             3| 10.0|
|    66688444476|             3| 40.0|
|    12345678966|             2|  7.5|
|    33344555587|             2| 10.0|
|    45345345376|             2| 20.0|
+---------------+--------------+-----+



In [47]:
spark.sql("""
SELECT * FROM empresa.trabalha where numero_projeto = 3 and cpf_funcionario in
(SELECT cpf_funcionario FROM empresa.trabalha where numero_projeto = 2)
""").show()

+---------------+--------------+-----+
|cpf_funcionario|numero_projeto|horas|
+---------------+--------------+-----+
|    33344555587|             3| 10.0|
+---------------+--------------+-----+



In [48]:
spark.sql("""
SELECT a.primeiro_nome, a.nome_meio, a.ultimo_nome
FROM empresa.funcionario a
INNER JOIN 
(SELECT * FROM empresa.trabalha where numero_projeto = 3 and cpf_funcionario in
(SELECT cpf_funcionario FROM empresa.trabalha where numero_projeto = 2)) b
ON a.cpf = b.cpf_funcionario
""").show()

+-------------+---------+-----------+
|primeiro_nome|nome_meio|ultimo_nome|
+-------------+---------+-----------+
|     Fernando|        T|       Wong|
+-------------+---------+-----------+



In [49]:
spark.sql("""
SELECT a.primeiro_nome, a.nome_meio, a.ultimo_nome
FROM empresa.projeto b
INNER JOIN empresa.trabalha c ON b.numero_projeto = c.numero_projeto
INNER JOIN empresa.funcionario a ON c.cpf_funcionario = a.cpf
where b.numero_departamento = 5
GROUP BY a.primeiro_nome, a.nome_meio, a.ultimo_nome
HAVING count(*) > 1
""").show()

+-------------+---------+-----------+
|primeiro_nome|nome_meio|ultimo_nome|
+-------------+---------+-----------+
|     Fernando|        T|       Wong|
+-------------+---------+-----------+



Numero dos projeto que contem um funcionario cujo o ultimo nome 'Souza', seja como funcionario ou gerente

In [50]:
spark.sql("""
SELECT  a.numero_projeto
FROM empresa.projeto a
INNER JOIN empresa.trabalha b ON a.numero_projeto = b.numero_projeto
INNER JOIN empresa.funcionario c ON b.cpf_funcionario = c.cpf
WHERE c.ultimo_nome = 'Souza'
UNION
SELECT  a.numero_projeto
FROM empresa.projeto a
INNER JOIN empresa.departamento b ON a.numero_departamento = b.numero_departamento
INNER JOIN empresa.funcionario c ON b.cpf_gerente = c.cpf
WHERE c.ultimo_nome = 'Souza'
""").show()

+--------------+
|numero_projeto|
+--------------+
|            30|
|            20|
|            10|
|             1|
+--------------+



Recuperar os nomes dos funcionarios que não possuem dependentes

In [51]:
spark.sql("""
SELECT a.primeiro_nome, a.nome_meio, a.ultimo_nome
FROM empresa.funcionario a
LEFT JOIN empresa.dependente b ON b.cpf_funcionario = a.cpf
where b.cpf_funcionario is null
""").show()

+-------------+---------+-----------+
|primeiro_nome|nome_meio|ultimo_nome|
+-------------+---------+-----------+
|        Joice|        A|      Leite|
|      Ronaldo|        K|       Lima|
|        André|        V|    Pereira|
|        Alice|        J|     Zelaya|
|        Jorge|        E|      Brito|
+-------------+---------+-----------+



Lista o nome dos gerentes que possuem pelo menos um dependente

In [52]:
spark.sql("""
SELECT distinct a.primeiro_nome, a.nome_meio, a.ultimo_nome
FROM empresa.funcionario a
INNER JOIN empresa.departamento c ON a.cpf = c.cpf_gerente
INNER JOIN empresa.dependente b ON b.cpf_funcionario = a.cpf
""").show()

+-------------+---------+-----------+
|primeiro_nome|nome_meio|ultimo_nome|
+-------------+---------+-----------+
|     Jeniffer|        S|      Souza|
|     Fernando|        T|       Wong|
+-------------+---------+-----------+

